In [67]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score,accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [68]:
trainData = pd.read_csv('Data/train_u6lujuX_CVtuZ9i.csv')

In [69]:
trainData.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [70]:
trainData.shape

(614, 13)

In [71]:
trainData.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [72]:
trainData.columns

Index([u'Loan_ID', u'Gender', u'Married', u'Dependents', u'Education',
       u'Self_Employed', u'ApplicantIncome', u'CoapplicantIncome',
       u'LoanAmount', u'Loan_Amount_Term', u'Credit_History', u'Property_Area',
       u'Loan_Status'],
      dtype='object')

In [73]:
#Handling with missing data

In [74]:
trainData.Gender.fillna(trainData.Gender.max(),inplace =True)

In [75]:
trainData.Married.fillna(trainData.Married.max(),inplace=True)

In [76]:
trainData.Credit_History.fillna(trainData.Credit_History.max(),inplace=True)

In [77]:
trainData.LoanAmount.fillna(trainData.LoanAmount.mean(),inplace=True)

In [78]:
trainData.Loan_Amount_Term.fillna(trainData.Loan_Amount_Term.mean(),inplace=True)

In [79]:
trainData.Self_Employed.fillna(trainData.Self_Employed.max(),inplace=True)

In [80]:
trainData.Dependents.fillna(0,inplace=True)

In [81]:
#Convert string values to numerical values because to algorithm can understand only numerical value not string values

In [82]:
trainData.Gender.value_counts()
gender_cat = pd.get_dummies(trainData.Gender,prefix='gender').gender_Female

In [83]:
trainData.Married.value_counts()
married_category = pd.get_dummies(trainData.Married,prefix='marriage').marriage_Yes

In [84]:
trainData.Education.value_counts()
graduate_category = pd.get_dummies(trainData.Education,prefix='education').education_Graduate

In [85]:
trainData.Self_Employed.value_counts()
self_emp_category = pd.get_dummies(trainData.Self_Employed,prefix='employed').employed_Yes

In [86]:
loan_status = pd.get_dummies(trainData.Loan_Status,prefix='status').status_Y

In [87]:
property_category = pd.get_dummies(trainData.Property_Area,prefix='property')

In [88]:
trainData.shape

(614, 13)

In [89]:
trainNew = pd.concat([trainData,gender_cat,married_category,graduate_category,self_emp_category,loan_status,property_category],axis=1)

In [90]:
trainNew.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,...,Property_Area,Loan_Status,gender_Female,marriage_Yes,education_Graduate,employed_Yes,status_Y,property_Rural,property_Semiurban,property_Urban
0,LP001002,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,...,Urban,Y,0,0,1,0,1,0,0,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,...,Rural,N,0,1,1,0,0,1,0,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,...,Urban,Y,0,1,1,1,1,0,0,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,...,Urban,Y,0,1,0,0,1,0,0,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,...,Urban,Y,0,0,1,0,1,0,0,1


In [91]:
trainNew.columns

Index([u'Loan_ID', u'Gender', u'Married', u'Dependents', u'Education',
       u'Self_Employed', u'ApplicantIncome', u'CoapplicantIncome',
       u'LoanAmount', u'Loan_Amount_Term', u'Credit_History', u'Property_Area',
       u'Loan_Status', u'gender_Female', u'marriage_Yes',
       u'education_Graduate', u'employed_Yes', u'status_Y', u'property_Rural',
       u'property_Semiurban', u'property_Urban'],
      dtype='object')

In [92]:
feature_columns = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','gender_Female','marriage_Yes','education_Graduate','employed_Yes','property_Rural','property_Semiurban','property_Urban']

In [93]:
X = trainNew[feature_columns]

In [162]:
y =  trainNew['status_Y']
y

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      0
10     1
11     1
12     1
13     0
14     1
15     1
16     1
17     0
18     0
19     1
20     0
21     1
22     0
23     0
24     0
25     1
26     1
27     1
28     0
29     1
      ..
584    0
585    0
586    1
587    1
588    1
589    0
590    1
591    0
592    1
593    1
594    1
595    1
596    0
597    0
598    1
599    1
600    0
601    1
602    1
603    1
604    1
605    0
606    1
607    1
608    1
609    1
610    1
611    1
612    1
613    0
Name: status_Y, dtype: uint8

In [123]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.01,random_state=42)

In [124]:
X_train.shape

(607, 12)

In [125]:
X_test.shape

(7, 12)

In [192]:
randForest = RandomForestClassifier(n_estimators=25, min_samples_split=25, max_depth=7, max_features=1)
randForest.fit(X_train,y_train)
y_pred_class  = randForest.predict(X_test)
randForestScore = accuracy_score(y_test,y_pred_class)
%time print "Random forest accuraccy score",randForestScore

Random forest accuraccy score 1.0
CPU times: user 333 µs, sys: 33 µs, total: 366 µs
Wall time: 297 µs


In [142]:
#Import test data and do real test of our model

In [190]:
randForestNew = RandomForestClassifier(n_estimators=25, min_samples_split=25, max_depth=7, max_features=1)
randForestNew.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=1, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=25, min_weight_fraction_leaf=0.0,
            n_estimators=25, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [144]:
testData = pd.read_csv('Data/test_Y3wMUE5_7gLdaTN.csv')

In [145]:
testData.shape

(367, 12)

In [146]:
testData.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [163]:
testData.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [148]:
testData.Gender.fillna(testData.Gender.max(),inplace =True)
testData.Married.fillna(testData.Married.max(),inplace=True)
testData.Credit_History.fillna(testData.Credit_History.max(),inplace=True)
testData.LoanAmount.fillna(testData.LoanAmount.mean(),inplace=True)
testData.Loan_Amount_Term.fillna(testData.Loan_Amount_Term.mean(),inplace=True)
testData.Self_Employed.fillna(testData.Self_Employed.max(),inplace=True)
testData.Dependents.fillna(0,inplace=True)

In [149]:
gender_cat = pd.get_dummies(testData.Gender,prefix='gender').gender_Female
married_category = pd.get_dummies(testData.Married,prefix='marriage').marriage_Yes
graduate_category = pd.get_dummies(testData.Education,prefix='education').education_Graduate
self_emp_category = pd.get_dummies(testData.Self_Employed,prefix='employed').employed_Yes
property_category = pd.get_dummies(testData.Property_Area,prefix='property')

In [150]:
testDataNew = pd.concat([testData,gender_cat,married_category,graduate_category,self_emp_category,property_category],axis=1)

In [151]:
X_testData = testDataNew[feature_columns]

In [152]:
X_testData.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,gender_Female,marriage_Yes,education_Graduate,employed_Yes,property_Rural,property_Semiurban,property_Urban
0,5720,0,110.0,360.0,1.0,0,1,1,0,0,0,1
1,3076,1500,126.0,360.0,1.0,0,1,1,0,0,0,1
2,5000,1800,208.0,360.0,1.0,0,1,1,0,0,0,1
3,2340,2546,100.0,360.0,1.0,0,1,1,0,0,0,1
4,3276,0,78.0,360.0,1.0,0,0,0,0,0,0,1


In [195]:
y_test_pread_class = randForestNew.predict(X_testData)

In [196]:
randForestFormat = ["Y" if i == 1 else "N" for i in y_test_pread_class ]

In [197]:
pd.DataFrame({'Loan_ID':testData.Loan_ID,'Loan_Status':randForestFormat}).to_csv('radom_forest_submission.csv',index=False)

In [ ]:
#Solve using logistic regression

In [183]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg.fit(X_train,y_train)
logREg_predict =logReg.predict(X_test)
accuracy_score(y_test,logREg_predict)

1.0

In [184]:
logReg_y_prediction_class = logReg.predict(X_testData)

In [185]:
logRegPredictionFormat = ["Y" if i == 1 else "N" for i in logReg_y_prediction_class ]

In [186]:
#zip(logRegPredictionFormat,logReg_y_prediction_class)

In [187]:
pd.DataFrame({'Loan_ID':testData.Loan_ID,'Loan_Status':logRegPredictionFormat}).to_csv('logReg_submission.csv',index=False)